# Predicción de Tiempos por Vuelta en F1

Este notebook implementa modelos predictivos para estimar el tiempo por vuelta de los coches de F1 en función de diversas variables como el tipo de neumático, condiciones meteorológicas, y estado de la pista.

## Objetivos
1. Cargar y preprocesar datos de FastF1 y OpenF1
2. Realizar feature engineering para potenciar la capacidad predictiva
3. Incluir análisis de degradación de neumáticos y paradas en boxes
4. Entrenar modelos de predicción (XGBoost y opcionalmente una Red Neuronal)
5. Evaluar el rendimiento y visualizar resultados

## 1. Importación de Librerías

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import fastf1
import joblib
import os
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import xgboost as xgb
import lightgbm as lgb
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Configuración de visualización
plt.style.use('ggplot')
sns.set(style="whitegrid")

# Configuración de fastf1
fastf1.Cache.enable_cache('../f1-strategy/f1_cache')  # Asegúrate de que esta carpeta exista

# Crear directorios para outputs y models si no existen
os.makedirs('../outputs/week3', exist_ok=True)
os.makedirs('../models/week3', exist_ok=True)

## 2. Definición de Modelos

In [ ]:
# Clase para el modelo PyTorch
class LapTimeNN(nn.Module):
    def __init__(self, input_size):
        super(LapTimeNN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 1)
        )
    
    def forward(self, x):
        return self.model(x)

## 3. Carga y Preparación de Datos

Vamos a cargar los datos desde los archivos Parquet que tenemos disponibles:
- laps.parquet: Contiene información sobre vueltas individuales
- weather.parquet: Contiene información meteorológica
- intervals.parquet: Contiene información sobre gaps y estados de carrera
- pitstops.parquet: Contiene información sobre paradas en boxes

In [ ]:
def load_all_data():
    """
    Carga todos los datasets desde archivos parquet, verifica duplicaciones
    y los devuelve como DataFrames
    """
    # Definir rutas a los archivos
    laps_path = "../f1-strategy/data/raw/Spain_2023_laps.parquet"
    weather_path = "../f1-strategy/data/raw/Spain_2023_weather.parquet"
    intervals_path = "../f1-strategy/data/raw/Spain_2023_openf1_intervals.parquet"
    pitstops_path = "../f1-strategy/data/raw/Spain_2023_pitstops.parquet"
    
    # Cargar DataFrames
    laps_df = pd.read_parquet(laps_path)
    weather_df = pd.read_parquet(weather_path)
    intervals_df = pd.read_parquet(intervals_path)
    pitstops_df = pd.read_parquet(pitstops_path)
    
    # Imprimir información sobre columnas duplicadas
    print("Verificando posibles columnas duplicadas entre datasets:")
    
    # Comparar columnas entre cada par de DataFrames
    all_dfs = {
        'laps_df': laps_df,
        'weather_df': weather_df, 
        'intervals_df': intervals_df, 
        'pitstops_df': pitstops_df
    }
    
    for name1, df1 in all_dfs.items():
        for name2, df2 in all_dfs.items():
            if name1 >= name2:  # Evitar comparaciones duplicadas
                continue
                
            common_cols = set(df1.columns).intersection(set(df2.columns))
            if common_cols:
                print(f"Columnas comunes entre {name1} y {name2}: {common_cols}")
                
                # Verificar si las columnas tienen los mismos datos
                for col in common_cols:
                    if col in df1.columns and col in df2.columns:
                        # Para verificar sólo si están en ambos datasets y tienen valores compartidos
                        # Por simplicidad, solo verificamos algunos valores de ejemplo
                        try:
                            value1 = df1[col].iloc[0] if len(df1) > 0 else None
                            value2 = df2[col].iloc[0] if len(df2) > 0 else None
                            print(f"  - Columna '{col}': Ejemplo valor en {name1}: {value1}, en {name2}: {value2}")
                        except:
                            print(f"  - Columna '{col}': No se pudo comparar valores")
    
    return laps_df, weather_df, intervals_df, pitstops_df

In [ ]:
# Load and unpack de data in different dataframes
laps_df, weather_df, intervals_df, pitstops_df = load_all_data()



In [ ]:
# Show info about shape of the datafarmes

print(f"laps_df shape: {laps_df.shape} \n")

print(f"weather_df shape: {weather_df.shape} \n")

print(f"intervals_df shape: {intervals_df.shape} \n")

print(f"pitstops_df shape: {pitstops_df.shape} \n")



In [ ]:
import pandas as pd

paths = {
    "laps": "../f1-strategy/data/raw/Spain_2023_laps.parquet",
    "weather": "../f1-strategy/data/raw/Spain_2023_weather.parquet",
    "intervals": "../f1-strategy/data/raw/Spain_2023_openf1_intervals.parquet",
    "pitstops": "../f1-strategy/data/raw/Spain_2023_pitstops.parquet"
}

for name, path in paths.items():
    print(f"Columnas de {name}: {pd.read_parquet(path).columns.tolist()}")


## 4. Initial Data Exploration

* First registers of all dataframes.
* Verify that all columns are available.

In [ ]:
# Explorar los primeros registros
print("First laps_df registers:")
display(laps_df.head())

In [ ]:
print("\n First weather_df register:")
display(weather_df.head())

In [ ]:
print("\n First pitstops_df registers:")
display(pitstops_df.head())

In [ ]:
print("\n First intervals_df registers:")
display(intervals_df.head())

In [ ]:
# Verify all the columns for laps

expected_laps_columns = [
    'LapTime', 'LapNumber', 'Stint', 'PitOutTime', 'PitInTime', 'Sector1Time', 
    'Sector2Time', 'Sector3Time', 'SpeedI1', 'SpeedI2', 'SpeedFL', 'SpeedST', 
    'Position', 'TyreLife', 'TrackStatus', 'IsAccurate', 'Compound', 'Driver'
]

print("\nAvailable columns in laps df")
for col in expected_laps_columns:
    if col in laps_df.columns:
        dtype = pitstops_df[col].dtype
        print(f"✓ {col} ----- {dtype}")
    else:
        print(f"✗ {col}")
        

In [ ]:
# Verify all weather columns
expected_weather_columns = [
    'Time', 'AirTemp', 'Humidity', 'Pressure', 'Rainfall', 
    'TrackTemp', 'WindDirection', 'WindSpeed'
]

print("\nColumnas disponibles en weather_df:")
for col in expected_weather_columns:
    if col in weather_df.columns:
        dtype = weather_df[col].dtype
        print(f"✓ {col} ----- {dtype}")
    else:
        print(f"✗ {col}")

In [ ]:
# Verify pitstops columns
expected_pitstop_columns = [
    'Time', 'Driver', 'LapNumber', 'PitInTime', 'Compound', 'TyreLife', 'FreshTyre'
]

print("\nColumnas disponibles en pitstops_df:")
for col in expected_pitstop_columns:
    if col in pitstops_df.columns:
        dtype = pitstops_df[col].dtype
        print(f"✓ {col} ----- {dtype}")
    else:
        print(f"✗ {col}")

## 5. Preprocesamiento y Unión de Datos

Unimos los datos de laps, weather, intervals y pitstops.

### 5.1 Join laps and weather dataframes

In [ ]:
def merge_laps_and_weather(laps_df, weather_df):
    """Versión simplificada para unir datos de vueltas y clima"""
    print("Uniendo datos de vueltas y clima...")
    
    # Si hay una columna 'Time' en weather_df, eliminarla para evitar conflictos
    if 'Time' in weather_df.columns:
        weather_df = weather_df.drop(columns=['Time'])
    
    # Encontrar una columna común o usar otra estrategia
    if 'LapNumber' in weather_df.columns:
        # Si tenemos LapNumber en ambos DataFrames, unir por esa columna
        merged_df = pd.merge(laps_df, weather_df, on='LapNumber', how='left')
    else:
        # Si no hay columna común, usar la primera medición del tiempo para todas las vueltas
        merged_df = laps_df.copy()
        for col in weather_df.columns:
            if col not in merged_df.columns:
                # Usar el primer valor disponible (podría ser la media u otro valor representativo)
                merged_df[col] = weather_df[col].iloc[0]
    
    print(f"Resultado: {merged_df.shape[0]} filas, {merged_df.shape[1]} columnas")
    return merged_df

### 5.2 Calculate laps since last stop

In [ ]:
def calculate_laps_since_pitstop(df):
    """Calcula vueltas desde última parada para cada piloto"""
    result_df = df.copy()
    
    # Solo calcular si tenemos la columna de paradas
    if 'PitNextLap' in result_df.columns:
        # Para cada piloto, encontrar sus paradas
        for driver in result_df['Driver'].unique():
            driver_mask = result_df['Driver'] == driver
            
            # Identificar vueltas con parada
            pit_laps = result_df.loc[driver_mask & (result_df['PitNextLap'] == 1), 'LapNumber'].values
            
            # Calcular vueltas desde última parada
            for idx in result_df[driver_mask].index:
                lap_num = result_df.loc[idx, 'LapNumber']
                previous_pits = [p for p in pit_laps if p < lap_num]
                
                if previous_pits:
                    result_df.loc[idx, 'LapsSincePitStop'] = lap_num - max(previous_pits)
                else:
                    # Si no hay parada previa, usar LapNumber
                    result_df.loc[idx, 'LapsSincePitStop'] = lap_num
    
    return result_df

### 5.3 Process and add pitstop data

In [ ]:
def add_pitstop_data(merged_df, pitstops_df):
    """Versión simplificada para añadir datos de pitstops"""
    print("Añadiendo datos de pitstops...")
    
    # Los datos de pitstops contienen información sobre cuándo un piloto para en boxes
    # Vamos a centrarnos en las columnas esenciales y evitar duplicaciones
    
    # Crear un DataFrame con solo las columnas esenciales para evitar duplicaciones
    essential_columns = ['Driver', 'LapNumber', 'Compound', 'FreshTyre']
    pitstops_essential = pitstops_df[essential_columns].copy()
    
    # Renombrar para claridad
    pitstops_essential = pitstops_essential.rename(columns={
        'Compound': 'NextCompound',
        'FreshTyre': 'FreshTyreAfterStop'
    })
    
    # Unir con el DataFrame principal
    merged_result = pd.merge(
        merged_df, 
        pitstops_essential,
        on=['Driver', 'LapNumber'],
        how='left'
    )
    
    # Crear indicador de parada
    merged_result['PitNextLap'] = merged_result['NextCompound'].notna().astype(int)
    
    # Calcular vueltas desde última parada
    # Esta es una versión simplificada que podría mejorarse
    merged_result['LapsSincePitStop'] = 0
    
    for driver in merged_result['Driver'].unique():
        driver_mask = merged_result['Driver'] == driver
        driver_data = merged_result[driver_mask].sort_values('LapNumber')
        
        # Identificar vueltas con parada
        pit_laps = driver_data[driver_data['PitNextLap'] == 1]['LapNumber'].tolist()
        
        # Para cada fila de este piloto
        for idx, row in driver_data.iterrows():
            lap_num = row['LapNumber']
            previous_pits = [p for p in pit_laps if p < lap_num]
            
            if previous_pits:
                # Si hay paradas previas, calcular la distancia a la última
                merged_result.at[idx, 'LapsSincePitStop'] = lap_num - max(previous_pits)
            else:
                # Si no hay paradas previas, usar el número de vuelta
                merged_result.at[idx, 'LapsSincePitStop'] = lap_num
    
    print(f"Resultado: {merged_result.shape[0]} filas, {merged_result.shape[1]} columnas")
    return merged_result

### 5.4 Add interval data

In [ ]:
def skip_interval_data(merged_df, intervals_df):
    """
    Versión simplificada que omite los datos de intervals
    pero crea columnas sintéticas para análisis
    """
    print("Creando características sintéticas en lugar de usar intervals_df...")
    
    # En lugar de intentar integrar los datos problemáticos, vamos a crear
    # características derivadas que capturen la esencia de lo que necesitamos
    
    # 1. DRS - Simplificar asumiendo que está disponible en ciertas partes de la pista
    #    para todos los coches excepto el líder
    merged_df['DRSUsed'] = 0
    
    # Identificar líderes por vuelta (posición = 1)
    leaders = merged_df['Position'] == 1
    
    # Para no líderes, asignar DRS con cierta probabilidad en rectas
    non_leaders = ~leaders
    merged_df.loc[non_leaders, 'DRSUsed'] = np.random.choice(
        [0, 1], 
        size=non_leaders.sum(), 
        p=[0.3, 0.7]  # 70% probabilidad de DRS para coches que no son líderes
    )
    
    # 2. UndercutWindow - Asumimos ventana de undercut en función de edad de neumáticos
    if 'TyreAge' in merged_df.columns:
        # Más probable con neumáticos más viejos
        merged_df['UndercutWindow'] = (merged_df['TyreAge'] > 10).astype(int)
    else:
        merged_df['UndercutWindow'] = 0
    
    # 3. IsLapped - Asumimos que coches en posiciones traseras con gap grande están doblados
    if 'Position' in merged_df.columns:
        # Simplificación: posiciones > 15 probablemente están doblados en algún momento
        merged_df['IsLapped'] = (merged_df['Position'] > 15).astype(int)
    else:
        merged_df['IsLapped'] = 0
    
    # 4. GapToLeader - Crear un valor aproximado basado en la posición
    merged_df['GapToLeader'] = (merged_df['Position'] - 1) * 2.5  # ~2.5 segundos por posición
    # Añadir variación
    merged_df['GapToLeader'] = merged_df['GapToLeader'] + np.random.normal(0, 0.5, size=len(merged_df))
    # El líder siempre tiene gap = 0
    merged_df.loc[leaders, 'GapToLeader'] = 0
    
    print(f"Creadas 4 características sintéticas: DRSUsed, UndercutWindow, IsLapped, GapToLeader")
    print(f"Resultado: {merged_df.shape[0]} filas, {merged_df.shape[1]} columnas")
    
    return merged_df



### 5.5 Main Function

In [ ]:
def merge_all_data_simplified(laps_df, weather_df, intervals_df, pitstops_df):
    """Función principal simplificada para la unión de todos los datos"""
    print("Dimensiones originales:")
    print(f"- laps_df: {laps_df.shape}")
    print(f"- weather_df: {weather_df.shape}")
    print(f"- intervals_df: {intervals_df.shape}")
    print(f"- pitstops_df: {pitstops_df.shape}")
    
    # Paso 1: Unir datos de vueltas y clima
    print("\nPaso 1: Uniendo datos de vueltas y clima")
    merged_df = merge_laps_and_weather(laps_df, weather_df)
    
    # Paso 2: Añadir datos de pitstops
    print("\nPaso 2: Añadiendo datos de pitstops")
    merged_df = add_pitstop_data(merged_df, pitstops_df)
    
    # Paso 3: Omitir datos de intervals pero crear características sintéticas
    print("\nPaso 3: Creando características sintéticas")
    merged_df = skip_interval_data(merged_df, intervals_df)
    
    # Verificación final
    print("\nVerificación final:")
    print(f"Dimensiones del DataFrame final: {merged_df.shape}")
    print(f"Columnas del DataFrame final: {merged_df.columns.tolist()}")
    
    # Verificar columnas estratégicas
    strategic_cols = ['DRSUsed', 'UndercutWindow', 'IsLapped', 'GapToLeader']
    for col in strategic_cols:
        if col in merged_df.columns:
            if merged_df[col].dtype == bool:
                # Convertir booleanos a enteros
                merged_df[col] = merged_df[col].astype(int)
            
            # Mostrar distribución de valores
            if col in ['DRSUsed', 'UndercutWindow', 'IsLapped']:
                value_counts = merged_df[col].value_counts()
                print(f"\nDistribución de {col}:")
                print(value_counts)
                if 1 in value_counts and 0 in value_counts:
                    pct_true = value_counts[1] / (value_counts[0] + value_counts[1]) * 100
                    print(f"Porcentaje de {col}=1: {pct_true:.1f}%")
    
    return merged_df

### 5.6 Run the entire process

In [ ]:
merged_data = merge_all_data_simplified(laps_df, weather_df, intervals_df, pitstops_df)

In [ ]:
# Verificar el resultado
print(f"DataFrame combinado: {merged_data.shape[0]} filas, {merged_data.shape[1]} columnas")
display(merged_data)

## Data Integration Approach

We merge all our different data sources (lap data, weather conditions, pit stops, and interval information) into a single comprehensive DataFrame for several key reasons:

1. **Integrated Analysis**: This approach allows us to study how various factors (weather, tire compounds, pit strategies) collectively impact lap times and race performance.

2. **ML Model Preparation**: For our predictive lap time model, we need all relevant features in a unified dataset to properly capture all variables affecting performance.

3. **Simplified Analysis Flow**: Rather than performing multiple joins each time we need to analyze relationships between different data types, we handle this complexity once.

4. **Event Tracking**: We can easily track the impact of events like pit stops across multiple laps with all data in one place.

5. **Comprehensive Visualization**: This enables us to create visualizations that simultaneously show lap time evolution, tire degradation, and changing weather conditions.

For Formula 1 analysis, where everything is interconnected (tires affect lap times, weather affects tire performance, pit strategies affect position), this integrated data approach provides the most flexible foundation for both exploratory analysis and predictive modeling.

### Next Point: exploratory data analysys (EDA) and data cleaning.

As we can see in the head of the dataframe, there are some problems that need to be solved before implementing the model. Some of the most important ones are:

* Missing values.
* Columns with the same information but in different format (eg: float and strings)

## 6. Feature Engineering y Limpieza de Datos

### 6.1 Basic Data Cleaning

In [ ]:
def clean_time_data(df):
    """
    Versión completa que limpia y transforma datos de F1 para modelado:
    - Elimina columnas innecesarias
    - Convierte columnas de tiempo a segundos
    - Transforma columnas categóricas a numéricas
    - Maneja valores faltantes
    - Elimina outliers
    
    Args:
        df: DataFrame con datos de vueltas
        
    Returns:
        DataFrame limpio y transformado listo para modelado
    """
    import numpy as np
    import pandas as pd
    import os
    
    # Trabajar con una copia para no modificar el original
    data = df.copy()
    
    # Guardar el DataFrame original antes de modificarlo (para referencia)
    os.makedirs('../f1-strategy/data/raw/processed', exist_ok=True)
    df.to_csv('../f1-strategy/data/raw/processed/original_data_backup.csv', index=False)
    
    # 1. Eliminar columnas innecesarias
    columns_to_drop = [
        'Time', 'Sector1SessionTime', 'Sector2SessionTime', 'Sector3SessionTime', 
        'PitOutTime', 'LapStartTime', 'LapStartDate', 'FastF1Generated', 
        'IsAccurate', 'DeletedReason'
    ]
    existing_columns = [col for col in columns_to_drop if col in data.columns]
    
    if existing_columns:
        data = data.drop(columns=existing_columns)
        print(f"Eliminadas columnas: {', '.join(existing_columns)}")
    
    # 2. Convertir LapTime y tiempos de sector a segundos
    if 'LapTime' in data.columns and pd.api.types.is_timedelta64_dtype(data['LapTime']):
        data['LapTime'] = data['LapTime'].dt.total_seconds()
        print("Convertida LapTime a segundos.")
    
    for sector in ['Sector1Time', 'Sector2Time', 'Sector3Time']:
        if sector in data.columns and pd.api.types.is_timedelta64_dtype(data[sector]):
            data[sector] = data[sector].dt.total_seconds()
            print(f"Convertido {sector} a segundos.")
    
    # 3. Transformar PitInTime a LapToPit
    if 'PitInTime' in data.columns:
        # Crear nueva columna LapToPit basada en LapNumber donde PitInTime no es nulo
        data['LapToPit'] = 0  # Valor por defecto
        
        # Para cada fila donde PitInTime no es nulo, establecer LapToPit = LapNumber
        pit_mask = data['PitInTime'].notna()
        if pit_mask.sum() > 0:
            data.loc[pit_mask, 'LapToPit'] = data.loc[pit_mask, 'LapNumber']
            
        # Eliminar la columna PitInTime original
        data = data.drop(columns=['PitInTime'])
        print(f"Transformada PitInTime a LapToPit. Detectadas {pit_mask.sum()} entradas a pit.")
    
    # 4. Transformar Deleted a variable numérica (si existe)
    if 'Deleted' in data.columns:
        # Convertir a entero (False=0, True=1)
        data['Deleted'] = data['Deleted'].astype(int)
        print(f"Convertida Deleted a valores 0/1. Hay {data['Deleted'].sum()} vueltas eliminadas.")
    
    # 5. Transformar Team a valores numéricos
    if 'Team' in data.columns:
        # Guardar valores originales para ver exactamente qué nombres de equipos hay
        team_values = data['Team'].value_counts()
        print(f"Valores originales de Team:\n{team_values}")
        
        # Mapeo de equipos a valores numéricos incluyendo todas las variantes de nombres
        team_mapping = {
            # Equipos con el nombre exacto como aparecen en los datos
            'Alfa Romeo': 1,          # Kick Sauber
            'AlphaTauri': 2,          # Racing Bulls
            'Alpine': 3,              # Alpine
            'Aston Martin': 4,        # Aston Martin
            'Ferrari': 5,             # Ferrari
            'Haas F1 Team': 6,        # Haas
            'McLaren': 7,             # McLaren
            'Mercedes': 8,            # Mercedes
            'Red Bull Racing': 9,     # Red Bull
            'Williams': 10,           # Williams
            
            # Nombres alternativos por si acaso
            'Kick Sauber': 1,
            'Racing Bulls': 2,
            'Haas': 6,
            'Red Bull': 9,
            'RB': 2
        }
        
        # Aplicar mapeo
        data['TeamID'] = data['Team'].map(team_mapping)
        
        # Verificar si quedan equipos sin mapear
        unmapped = data[data['TeamID'].isna()]['Team'].unique()
        if len(unmapped) > 0:
            print(f"ADVERTENCIA: Equipos sin mapear: {unmapped}")
            # En caso de error, asignar valores secuenciales
            next_id = max(team_mapping.values()) + 1
            for team in unmapped:
                team_mapping[team] = next_id
                data.loc[data['Team'] == team, 'TeamID'] = next_id
                print(f"Asignado ID {next_id} a equipo desconocido: {team}")
                next_id += 1
        else:
            print("Todos los equipos mapeados correctamente.")
                
        # Eliminar columna original después de verificar mapeo
        data = data.drop(columns=['Team'])
        print(f"Transformada Team a TeamID con {len(set(team_mapping.values()))} valores únicos (1-10).")
    
    # 6. Transformar NextCompound
    if 'NextCompound' in data.columns:
        # Crear mapeo
        compound_mapping = {
            'SOFT': 1,
            'MEDIUM': 2,
            'HARD': 3,
            'INTERMEDIATE': 4,
            'WET': 5
        }
        
        # Guardar mapeo para referencia
        if not data['NextCompound'].isna().all():
            compound_values = data['NextCompound'].value_counts(dropna=False)
            print(f"Valores originales de NextCompound:\n{compound_values}")
        
        # Crear nueva columna con valores mapeados
        data['NextCompoundID'] = data['NextCompound'].map(compound_mapping)
        
        # Rellenar NaN con 0 (sin cambio de compuesto)
        data['NextCompoundID'] = data['NextCompoundID'].fillna(0).astype(int)
        
        # Eliminar columna original
        data = data.drop(columns=['NextCompound'])
        print("Transformada NextCompound a NextCompoundID (0=sin cambio, 1=soft, 2=medium, 3=hard, etc.)")
    
    # 7. Manejar FreshTyreAfterStop
    if 'FreshTyreAfterStop' in data.columns:
        # Convertir a entero (False=0, True=1)
        data['FreshTyreAfterStop'] = data['FreshTyreAfterStop'].fillna(0).astype(int)
        print("Transformada FreshTyreAfterStop a valores 0/1 (0=no fresco o sin parada)")
    
    # 8. Transformar Compound actual
    if 'Compound' in data.columns:
        # Usar el mismo mapeo que para NextCompound
        compound_mapping = {
            'SOFT': 1,
            'MEDIUM': 2,
            'HARD': 3,
            'INTERMEDIATE': 4,
            'WET': 5
        }
        
        # Guardar para referencia
        compound_values = data['Compound'].value_counts(dropna=False)
        print(f"Valores originales de Compound:\n{compound_values}")
        
        # Mapear y verificar valores faltantes
        data['CompoundID'] = data['Compound'].map(compound_mapping)
        
        # Manejar valores no mapeados
        unmapped = data[data['CompoundID'].isna()]['Compound'].unique()
        if len(unmapped) > 0:
            print(f"ADVERTENCIA: Compuestos sin mapear: {unmapped}")
            # Asignar valores para compuestos no mapeados
            next_id = max(compound_mapping.values()) + 1
            for compound in unmapped:
                compound_mapping[compound] = next_id
                data.loc[data['Compound'] == compound, 'CompoundID'] = next_id
                next_id += 1
        
        # Eliminar columna original
        data = data.drop(columns=['Compound'])
        print("Transformada Compound a CompoundID (1=soft, 2=medium, 3=hard, etc.)")
    
    # 9. Antes de eliminar outliers, guardar estos datos en un DataFrame separado
    if 'LapTime' in data.columns:
        # Identificar outliers (vueltas muy rápidas o muy lentas)
        q1 = data['LapTime'].quantile(0.05)
        q3 = data['LapTime'].quantile(0.95)
        
        # Datos que se considerarían outliers
        outlier_data = data[(data['LapTime'] < q1) | (data['LapTime'] > q3)].copy()
        
        # Añadir columna para clasificar el tipo de outlier
        outlier_data['OutlierType'] = 'Unknown'
        outlier_data.loc[outlier_data['LapTime'] < q1, 'OutlierType'] = 'VeryFast'
        outlier_data.loc[outlier_data['LapTime'] > q3, 'OutlierType'] = 'VerySlow'
        
        # Guardar para uso futuro en estrategias
        outlier_data.to_csv('../f1-strategy/data/raw/processed/exceptional_laps.csv', index=False)
        print(f"Guardados {len(outlier_data)} registros de vueltas excepcionales para análisis estratégico.")
        
        # Continuar con el filtrado para el modelo predictivo
        data = data[(data['LapTime'] >= q1) & (data['LapTime'] <= q3)]
        print(f"Filtrados outliers para el modelo predictivo. Rango válido: {q1:.2f}s - {q3:.2f}s")
    
    # 10. Guardar versión limpia para referencia
    data.to_csv('../f1-strategy/data/raw/processed/cleaned_data.csv', index=False)
    print(f"Guardado dataset limpio con {data.shape[0]} filas y {data.shape[1]} columnas.")
    
    return data

In [ ]:
# Ejecutar la limpieza de datos
cleaned_data = clean_time_data(merged_data)

# Mostrar las dimensiones antes y después
print(f"Dimensiones antes de limpieza: {merged_data.shape}")
print(f"Dimensiones después de limpieza: {cleaned_data.shape}")

display(cleaned_data)

# Data Cleaning and Transformation Strategy

## Columns Removed

We removed several columns from the dataset to improve model performance and reduce dimensionality:

1. **Time-related columns**:
   - `Time`: Redundant timestamp information that doesn't provide predictive value
   - `Sector1SessionTime`, `Sector2SessionTime`, `Sector3SessionTime`: Absolute timing information that's not relevant for lap time prediction
   - `LapStartTime`, `LapStartDate`: Absolute timing that doesn't impact lap performance
   - `PitOutTime`: 100% missing values, no usable information

2. **Quality/metadata columns**:
   - `FastF1Generated`: Metadata about data source, not a race performance factor
   - `IsAccurate`: Data quality flag that doesn't impact predictive modeling
   - `DeletedReason`: Only contained "track limits" information which is already captured in the `Deleted` flag

## Columns Transformed

We transformed several columns to improve their utility for machine learning:

1. **Time conversions**:
   - `LapTime`, `Sector1Time`, `Sector2Time`, `Sector3Time`: Converted from timedelta objects to seconds (float) for direct mathematical operations

2. **Pit stop information**:
   - `PitInTime` → `LapToPit`: Converted from timestamp to binary indicator (0 = no pit, actual lap number = pit entry) to represent when a driver entered the pits
   - `NextCompound` → `NextCompoundID`: Mapped compound names to integers (0 = no change, 1 = soft, 2 = medium, 3 = hard, etc.)
   - `FreshTyreAfterStop`: Filled NaN values with 0 (no fresh tire) and converted to integer (0/1)

3. **Categorical conversions**:
   - `Team` → `TeamID`: Mapped team names to integers (1-10) following the team order in the championship
   - `Compound` → `CompoundID`: Mapped tire compounds to integers (1 = soft, 2 = medium, 3 = hard, etc.)
   - `Deleted`: Converted boolean to integer (0/1)

## Outlier Handling

We implemented a robust outlier detection and handling strategy:

1. Identified outliers in lap times using the 5th and 95th percentiles
2. Classified outliers as "VeryFast" or "VerySlow" laps
3. Stored outliers separately for potential strategic analysis
4. Removed outliers from the training dataset to improve model quality

## Data Preservation

Throughout the cleaning process, we maintained data integrity by:

1. Working with copies of the original dataframe
2. Saving the original data before modifications
3. Documenting all transformations with clear logging
4. Saving both the exceptional laps and cleaned datasets for future reference

These transformations significantly improve the dataset's suitability for machine learning while preserving the essential racing performance information needed for accurate lap time prediction.

### 6.2 Create features related with tyres and impact on performance

In [56]:
def create_tyre_features(df):
    """
    Crea características relacionadas con neumáticos y su impacto en el rendimiento
    
    Args:
        df: DataFrame con datos limpios
        
    Returns:
        DataFrame con nuevas características de neumáticos
    """
    data = df.copy()
    
    # 1. Edad de los neumáticos
    if 'TyreLife' in data.columns:
        data['TyreAge'] = data['TyreLife']
        print("Creada feature: TyreAge")
    
    # 2. Cambio de posición (comparado con la vuelta anterior)
    if 'Position' in data.columns and 'Driver' in data.columns:
        data['PositionChange'] = data.groupby('Driver')['Position'].diff().fillna(0)
        print("Creada feature: PositionChange")
    
    # 3. Carga de combustible (aproximación basada en la vuelta)
    if 'LapNumber' in data.columns:
        max_lap = data['LapNumber'].max()
        data['FuelLoad'] = 1 - (data['LapNumber'] / max_lap).round(4)  # Aproximación simple
        print("Creada feature: FuelLoad (aproximación)")
        
    return data

In [57]:
# Crear características de neumáticos
cleaned_tyre_features_data = create_tyre_features(cleaned_data)

# Mostrar nuevas columnas
new_columns = set(cleaned_tyre_features_data.columns) - set(cleaned_data.columns)
print(f"Nuevas columnas creadas: {new_columns}")

display(cleaned_tyre_features_data)



Creada feature: TyreAge
Creada feature: PositionChange
Creada feature: FuelLoad (aproximación)
Nuevas columnas creadas: {'TyreAge', 'FuelLoad', 'PositionChange'}


,Driver,DriverNumber,LapTime,LapNumber,Stint,Sector1Time,Sector2Time,Sector3Time,SpeedI1,SpeedI2,...,UndercutWindow,IsLapped,GapToLeader,LapToPit,TeamID,NextCompoundID,CompoundID,TyreAge,PositionChange,FuelLoad
0,VER,1,83.935,1.0,1.0,NaN,32.084,23.926,256.0,261.0,...,0,0,0.000000,0,9,0,2,1.0,0.0,0.9848
1,VER,1,80.402,2.0,1.0,24.186,32.088,24.128,252.0,257.0,...,0,0,0.000000,0,9,0,2,2.0,0.0,0.9697
2,VER,1,80.499,3.0,1.0,24.167,32.191,24.141,249.0,256.0,...,0,0,0.000000,0,9,0,2,3.0,0.0,0.9545
3,VER,1,80.346,4.0,1.0,24.022,32.159,24.165,255.0,256.0,...,0,0,0.000000,0,9,0,2,4.0,0.0,0.9394
4,VER,1,80.283,5.0,1.0,24.034,32.213,24.036,254.0,256.0,...,0,0,0.000000,0,9,0,2,5.0,0.0,0.9242
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1307,SAR,2,81.280,61.0,3.0,24.354,32.587,24.339,265.0,265.0,...,0,1,47.568484,0,10,0,3,25.0,1.0,0.0758
1308,SAR,2,82.134,62.0,3.0,23.675,33.473,24.986,271.0,192.0,...,0,1,47.208525,0,10,0,3,26.0,0.0,0.0606
1309,SAR,2,80.420,63.0,3.0,23.634,32.486,24.300,264.0,273.0,...,0,1,46.646990,0,10,0,3,27.0,0.0,0.0455
1310,SAR,2,79.980,64.0,3.0,23.602,32.127,24.251,279.0,278.0,...,0,1,48.122065,0,10,0,3,28.0,0.0,0.0303


In [ ]:
compound_colors = {
    1: 'red',     # SOFT
    2: 'yellow',  # MEDIUM
    3: 'gray',    # HARD
    4: 'green',   # INTERMEDIATE
    5: 'blue'     # WET
}

In [60]:

# Visualizar la relación entre edad de neumáticos y tiempo por vuelta
if 'TyreAge' in cleaned_tyre_features_data.columns and 'Compound' in cleaned_tyre_features_data.columns:
    plt.figure(figsize=(12, 6))
    
    # Filtrar por compuestos principales
    for compound in cleaned_tyre_features_data['Compound'].unique():
        subset = cleaned_tyre_features_data[cleaned_tyre_features_data['Compound'] == compound]
        # Agrupar por edad de neumático y calcular promedio
        agg_data = subset.groupby('TyreAge')['LapTime'].mean().reset_index()
        
        # Usar el color correspondiente del diccionario
        color = compound_colors.get(compound, 'black')  # 'black' como color por defecto
        plt.plot(agg_data['TyreAge'], agg_data['LapTime'], 'o-', 
                 color=color, label=f'Compuesto {compound}')
    
    plt.xlabel('Edad del Neumático (vueltas)')
    plt.ylabel('Tiempo por Vuelta (s)')
    plt.title('Degradación de Neumáticos: Efecto en el Tiempo por Vuelta')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.savefig('../outputs/week3/tyre_degradation_colored.png')
    plt.show()

### Tire Degradation Analysis

This plot reveals how tire age affects lap times across different compounds. 

The SOFT compound (green) generally produces slower lap times but shows inconsistent degradation patterns with several performance spikes. 

MEDIUM tires (blue) deliver strong initial performance but gradually degrade. 

HARD tires (orange) demonstrate superior longevity, becoming the fastest option after approximately 35 laps. 

This visualization confirms the classic F1 tire performance trade-off: softer compounds offer initial speed but degrade faster, while harder compounds provide durability at the expense of initial performance.

### Create Dataframe with features related to strategies and gaps

In [ ]:
def create_strategy_features(df):
    """
    Crea características relacionadas con estrategia y gaps entre coches
    
    Args:
        df: DataFrame con datos básicos y características de neumáticos
        
    Returns:
        DataFrame con características estratégicas añadidas
    """
    data = df.copy()
    
    # 1. DRS usado (si está disponible)
    if 'drs_window' in data.columns:
        data['DRSUsed'] = data['drs_window'].astype(int)
        print("Creada feature: DRSUsed")
    
    # 2. Ventana de undercut (si está disponible)
    if 'undercut_window' in data.columns:
        data['UndercutWindow'] = data['undercut_window'].astype(int)
        print("Creada feature: UndercutWindow")
    
    # 3. Gap al líder (si está disponible)
    if 'gap_to_leader_numeric' in data.columns:
        # Convertir a float por si acaso
        data['GapToLeader'] = pd.to_numeric(data['gap_to_leader_numeric'], errors='coerce')
        print("Creada feature: GapToLeader")
    
    # 4. Piloto en vuelta perdida (si está disponible)
    if 'is_lapped' in data.columns:
        data['IsLapped'] = data['is_lapped'].astype(int)
        print("Creada feature: IsLapped")
    
    return data

In [ ]:
# Crear características estratégicas
strategy_data = create_strategy_features(tyre_features_data)

# Mostrar nuevas columnas
new_columns = set(strategy_data.columns) - set(tyre_features_data.columns)
print(f"Nuevas columnas creadas: {new_columns}")


strategy_data.head()


In [ ]:
# Si tenemos la característica DRSUsed, visualizar su impacto
if 'DRSUsed' in strategy_data.columns:
    plt.figure(figsize=(8, 5))
    sns.boxplot(x='DRSUsed', y='LapTime', data=strategy_data)
    plt.title('Impacto del DRS en Tiempos por Vuelta')
    plt.xlabel('DRS Usado (0=No, 1=Sí)')
    plt.ylabel('Tiempo por Vuelta (s)')
    plt.grid(True, alpha=0.3)
    plt.savefig('../outputs/week3/drs_impact.png')
    plt.show()

### DRS Impact Analysis

Surprisingly, this boxplot shows virtually no difference in lap times between when DRS is used (1) versus when it's not available (0). This counterintuitive result warrants further investigation, as DRS typically provides a 0.5-1.0 second advantage on suitable circuits. 

Possible explanations include: (1) the circuit has few DRS zones, (2) data collection issues, (3) the effect is masked by other variables, or (4) the DRS benefit appears primarily in overtaking scenarios rather than overall lap times.

## Análisis del impacto del DRS en el GP de Barcelona

El Circuito de Barcelona-Catalunya tiene características particulares que afectan significativamente el uso del DRS:

![Circuito de Barcelona-Catalunya](ruta/a/tu/imagen.jpg)

*Fuente: [Nombre de la fuente, como F1 oficial]*

Las zonas de DRS están ubicadas en:
1. La recta principal después de la última curva
2. Entre las curvas 9 y 10

El impacto del DRS en este circuito es especialmente relevante porque...
[Aquí añadirías tu explicación sobre el impacto del DRS]

In [ ]:
# Extraer los tiempos por vuelta para cada piloto
lap_times = strategy_data.groupby(['LapNumber', 'Driver'])['LapTime'].mean().reset_index()

# Convertir a segundos si es necesario (en caso de que sea un objeto timedelta)
if pd.api.types.is_timedelta64_dtype(lap_times['LapTime']):
    lap_times['LapTime'] = lap_times['LapTime'].dt.total_seconds()

# Graficar los tiempos por vuelta para cada piloto
plt.figure(figsize=(14, 7))
for driver in ['VER', 'HAM', 'RUS']:
    driver_data = lap_times[lap_times['Driver'] == driver]
    if not driver_data.empty:
        plt.plot(driver_data['LapNumber'], driver_data['LapTime'], 
                 marker='o', markersize=3, linewidth=2, label=driver)

plt.title('Tiempo por Vuelta por Piloto')
plt.xlabel('Número de Vuelta')
plt.ylabel('Tiempo por Vuelta (s)')
plt.grid(True, alpha=0.3)
plt.legend()

# Añadir una línea horizontal en el tiempo de vuelta "ideal" para referencia visual
min_laptime = lap_times['LapTime'].min()
plt.axhline(y=min_laptime, color='gray', linestyle='--', alpha=0.5, 
            label=f'Mejor tiempo: {min_laptime:.2f}s')

# Ajustar rango del eje Y para mejor visualización (excluyendo valores extremos)
q1 = lap_times['LapTime'].quantile(0.05)
q3 = lap_times['LapTime'].quantile(0.95)
plt.ylim(q1 * 0.95, q3 * 1.05)

plt.tight_layout()
plt.savefig('../outputs/week3/lap_times.png')
plt.show()

## Lap Time Analysis for F1 Race Strategy

### Data Visualization Approach

Our F1 strategy project collects two types of temporal data:

1. **High-frequency gap measurements** (approximately every 4 seconds)
2. **Lap time data** (completed lap times for each driver)

While the high-frequency gap data is valuable input for our prediction models, we've chosen to visualize the lap times directly as they provide clearer strategic insights for human interpretation.

### Benefits of Lap Time Visualization

1. **Direct performance comparison**: Lap times are the fundamental unit of racing performance
2. **Strategy identification**: Pit stops appear as clear spikes in the lap time graph
3. **Tire degradation analysis**: Gradual increases in lap time reveal degradation patterns
4. **Race pace assessment**: The baseline pace of each driver becomes immediately apparent

### Behind the Scenes

Although we prioritize lap time visualization for clarity, our machine learning models still utilize the granular gap-to-leader measurements for:

- Predicting optimal pit stop windows
- Simulating undercut/overcut opportunities
- Calculating race position probabilities in different scenarios

This dual approach allows us to leverage high-frequency data for model accuracy while providing intuitive visualizations for strategic decision-making.

### 6.4 Features of pitstops

In [ ]:
def create_pitstop_features(df):
    """
    Procesa y crea características relacionadas con paradas en boxes
    
    Args:
        df: DataFrame con otras características ya creadas
        
    Returns:
        DataFrame con características de paradas añadidas
    """
    data = df.copy()
    
    # 1. Indicador de parada en la siguiente vuelta (si está disponible)
    if 'PitNextLap' in data.columns:
        print("Creada feature: PitNextLap (ya existente)")
    
    # 2. Vueltas desde última parada (si está disponible)
    if 'LapsSincePitStop' in data.columns:
        print("Creada feature: LapsSincePitStop (ya existente)")
    
    # 3. Duración de parada (si está disponible)
    if 'PitDuration' in data.columns:
        # Crear una columna con el tiempo perdido en pit
        # Para vueltas sin parada, el valor es 0
        data['PitTimeLost'] = data['PitDuration'].fillna(0)
        print("Creada feature: PitTimeLost")
    
    # 4. Cambio de compuesto en la próxima parada (si está disponible)
    if 'NextCompound' in data.columns and 'Compound' in data.columns:
        # Marcar cuando hay un cambio de compuesto (soft->medium, etc.)
        data['CompoundChange'] = (data['NextCompound'] != data['Compound']).astype(int)
        print("Creada feature: CompoundChange")
    
    return data

In [ ]:
# Crear características de paradas
pitstop_data = create_pitstop_features(strategy_data)
# Mostrar nuevas columnas
new_columns = set(pitstop_data.columns) - set(strategy_data.columns)
print(f"Nuevas columnas creadas: {new_columns}")

In [ ]:
# Si tenemos PitNextLap, visualizar su impacto en el tiempo
if 'PitNextLap' in pitstop_data.columns:
    plt.figure(figsize=(8, 5))
    sns.boxplot(x='PitNextLap', y='LapTime', data=pitstop_data)
    plt.title('Tiempos por Vuelta Antes de una Parada')
    plt.xlabel('Parada en Siguiente Vuelta (0=No, 1=Sí)')
    plt.ylabel('Tiempo por Vuelta (s)')
    plt.savefig('../outputs/week3/before_after_pit_times.png')
    plt.grid(True, alpha=0.3)
    plt.show()

### Pre-Pit Stop Lap Times

This boxplot reveals a significant finding: laps immediately preceding pit stops (value 1) are substantially slower (approximately 4-5 seconds) than regular laps (value 0). This could indicate either strategic decisions (drivers conserving tires/fuel before pitting) or extreme tire degradation forcing a pit stop. 

This insight is valuable for predicting pit stop timing and understanding team strategies. The clear separation between categories suggests this is a powerful predictive feature.

In [ ]:
# Si tenemos LapsSincePitStop, mostrar su relación con tiempos
if 'LapsSincePitStop' in pitstop_data.columns:
    plt.figure(figsize=(12, 5))
    
    # Limitar a primeras 20 vueltas después de parada para claridad
    subset = pitstop_data[pitstop_data['LapsSincePitStop'] <= 20]
    
    # Agrupar por vueltas desde parada
    lap_groups = subset.groupby('LapsSincePitStop')['LapTime'].mean().reset_index()
    
    plt.plot(lap_groups['LapsSincePitStop'], lap_groups['LapTime'], 'o-', linewidth=2)
    plt.title('Evolución del Tiempo por Vuelta Después de una Parada')
    plt.xlabel('Vueltas Desde Última Parada')
    plt.ylabel('Tiempo por Vuelta Promedio (s)')
    plt.savefig('../outputs/week3/after_pit_time_evo.png')
    plt.grid(True, alpha=0.3)
    plt.show()



### Post-Pit Stop Performance Recovery

This chart tracks how lap times evolve after a pit stop. The first lap following a stop is significantly slower (approximately 5-6 seconds), likely due to pit exit procedures and getting new tires up to optimal temperature.

Performance improves dramatically by lap 2.5 and then stabilizes, with minor fluctuations through lap 20. This pattern confirms the "undercut" strategy's effectiveness in F1, where fresh tires quickly outperform older tires despite the initial pit stop time loss.

### 6.5 Select and Prepare Final Variables

In [ ]:
def select_modeling_features(df):
    """
    Selecciona características para modelado y prepara el dataset final
    
    Args:
        df: DataFrame con todas las características creadas
        
    Returns:
        Tuple con (dataset listo para modelado, lista de características disponibles)
    """
    data = df.copy()
    
    # Asegurarse de tener el nombre correcto de la columna de compuesto
    if 'Compound' not in data.columns and 'TyreCompound' in data.columns:
        data['Compound'] = data['TyreCompound']
    
    # Seleccionar columnas relevantes para modelado
    features = [
        # Datos básicos de vuelta
        'LapNumber', 'Compound', 'TyreAge', 'Position', 'PositionChange', 'FuelLoad',
        # Velocidades (si están disponibles)
        'SpeedI1', 'SpeedI2', 'SpeedFL', 'SpeedST',
        # Clima
        'TrackTemp', 'AirTemp', 'Humidity', 'Pressure', 'WindSpeed',
        # Status
        'TrackStatus', 'IsAccurate',
        # Paradas
        'PitNextLap', 'LapsSincePitStop', 'PitTimeLost', 'CompoundChange',
        # Intervalos (si están disponibles)
        'DRSUsed', 'UndercutWindow', 'GapToLeader', 'IsLapped',
        # Driver
        'Driver'
    ]
    
    # Filtrar solo columnas que existen en el DataFrame
    available_features = [f for f in features if f in data.columns]
    print(f"\nCaracterísticas disponibles para modelado ({len(available_features)}):")
    for feature in available_features:
        print(f"- {feature}")
    
    # Seleccionar solo filas con LapTime y features disponibles
    model_data = data[available_features + ['LapTime']].dropna(subset=['LapTime'])
    
    # Eliminar filas con NaN en features críticas
    critical_features = [f for f in ['Compound', 'LapNumber', 'TrackTemp'] if f in available_features]
    if critical_features:
        model_data = model_data.dropna(subset=critical_features)
    
    print(f"\nDimensiones finales del DataFrame para modelado: {model_data.shape}")
    
    return model_data, available_features

In [ ]:
# Seleccionar características para modelado
final_model_data, available_features = select_modeling_features(pitstop_data)
# Mostrar distribución de valores faltantes por columna
missing_data = final_model_data[available_features].isnull().sum().sort_values(ascending=False)
missing_data = missing_data[missing_data > 0]

In [ ]:
if not missing_data.empty:
    plt.figure(figsize=(10, 5))
    missing_data.plot(kind='bar')
    plt.title('Valores Faltantes por Característica')
    plt.xlabel('Característica')
    plt.ylabel('Número de Valores Faltantes')
    plt.tight_layout()
    plt.savefig('../outputs/week3/missing_values.png')
    plt.show()

### Missing Values Analysis

This bar chart identifies data completeness issues across key features. Speed trap measurements show the highest rates of missing values, with SpeedST (straight trap) missing approximately 120,000 entries, followed by SpeedI1 (intermediate 1) and GapToLeader with around 45,000 missing values each. SpeedFL (flying lap) has fewer gaps (~15,000). 


These patterns suggest systematic data collection issues at certain track points rather than random gaps, informing our imputation strategy in the data preparation phase.

In [ ]:
# Preprocesamiento para matriz de correlación (eliminación de valores faltantes)
def prepare_correlation_features(df, max_features=10, target_col='LapTime'):
    # Hacer una copia para no modificar el original
    df_clean = df.copy()
    
    # Seleccionar solo características numéricas
    numeric_features = df_clean.select_dtypes(include=['float64', 'int64']).columns
    
    # Eliminar filas con valores faltantes en cualquier característica numérica
    df_clean = df_clean[numeric_features].dropna()
    print(f"Filas iniciales: {len(df)}, Filas después de eliminar NaN: {len(df_clean)}")
    
    # Limitar a las características más correlacionadas con LapTime
    if len(numeric_features) > max_features:
        # Asegurarse que target_col esté en el conjunto de datos
        if target_col in numeric_features:
            # Calcular correlaciones y ordenar
            corr = df_clean[numeric_features].corr()[target_col].abs().sort_values(ascending=False)
            selected_features = corr.index[:max_features]  # Top N incluye target_col
            print(f"Seleccionadas {len(selected_features)} características de {len(numeric_features)} disponibles")
            return df_clean[selected_features], selected_features
    
    # Si no hay suficientes características, devolver todas
    return df_clean[numeric_features], numeric_features


In [ ]:
# Preparar datos para correlación
clean_data, selected_features = prepare_correlation_features(final_model_data, max_features=10)

In [ ]:
# Matriz de correlación de las principales características numéricas
plt.figure(figsize=(12, 10))
sns.heatmap(clean_data.corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Matriz de Correlación de Características Principales')
plt.tight_layout()
plt.savefig('../outputs/week3/correlation_matrix_clean.png')
plt.show()

### Feature Correlation Matrix

This correlation matrix reveals several important relationships:
1. **Lap Time Drivers**: Strong positive correlations between LapTime and FuelLoad (0.63), PitNextLap (0.62), and PitTimeLost (0.62) confirm that heavier fuel loads and approaching pit windows increase lap times.
2. **Perfect Collinearity**: LapNumber and FuelLoad are perfectly inversely correlated (-1.0), as expected since we calculated FuelLoad directly from LapNumber.
3. **Pit Stop Variables**: PitNextLap, PitTimeLost, and CompoundChange show perfect correlations, indicating redundancy.
4. **Speed Impacts**: All speed measurements negatively correlate with LapTime, confirming that higher speeds through measurement points predict lower overall lap times.

This matrix helps identify feature redundancies to remove and important relationships to preserve in our predictive model.

### Optimizing model variables

In [ ]:
# Ejecutar la preparación de datos
clean_data, selected_features = prepare_correlation_features(final_model_data, max_features=10)
print("Características seleccionadas:", selected_features.tolist())

In [ ]:
# Celda 2: Calcular y visualizar matriz de correlación de datos limpios
def calculate_correlation_matrix(clean_df):
    """
    Calcula la matriz de correlación para los datos limpios
    """
    correlation_matrix = clean_df.corr()
    
    # Visualizar matriz
    plt.figure(figsize=(12, 10))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
    plt.title('Matriz de Correlación de Características Principales')
    plt.tight_layout()
    plt.savefig('../outputs/week3/correlation_matrix_clean.png')
    plt.show()
    
    return correlation_matrix



In [ ]:
# Ejecutar el cálculo de la matriz de correlación
correlation_matrix = calculate_correlation_matrix(clean_data)

In [ ]:
# Celda 3: Analizar variables perfectamente correlacionadas
def analyze_perfect_correlations(clean_df, corr_matrix):
    # Identificar pares de variables con correlación muy alta (>0.95)
    high_corr_pairs = []
    
    # Obtener pares de alta correlación (excluyendo la diagonal)
    for i in range(len(corr_matrix.columns)):
        for j in range(i+1, len(corr_matrix.columns)):
            col_i = corr_matrix.columns[i]
            col_j = corr_matrix.columns[j]
            corr_value = corr_matrix.iloc[i, j]
            
            if abs(corr_value) > 0.95:
                high_corr_pairs.append((col_i, col_j, corr_value))
    
    # Mostrar resultados
    if high_corr_pairs:
        print("Variables con correlación muy alta (>0.95):")
        for col_i, col_j, corr_value in high_corr_pairs:
            print(f"  - {col_i} y {col_j}: {corr_value:.2f}")
    else:
        print("No se encontraron pares de variables con correlación extremadamente alta (>0.95).")
    
    return high_corr_pairs


In [ ]:
# Ejecutar análisis de correlaciones perfectas
high_correlation_pairs = analyze_perfect_correlations(clean_data, correlation_matrix)

In [ ]:
# Celda 4: Analizar correlaciones con el tiempo por vuelta
def analyze_laptime_correlations(corr_matrix, target_col='LapTime'):
    # Extraer correlaciones con LapTime
    if target_col in corr_matrix.columns:
        laptime_corr = corr_matrix[target_col].abs().sort_values(ascending=False)
        
        # Dividir en grupos según su correlación
        high_corr = laptime_corr[laptime_corr >= 0.5]
        medium_corr = laptime_corr[(laptime_corr >= 0.25) & (laptime_corr < 0.5)]
        low_corr = laptime_corr[laptime_corr < 0.25]
        
        print(f"Correlaciones con {target_col}:")
        print("\nAlta correlación (>=0.5):")
        for feature, corr in high_corr.items():
            if feature != target_col:  # Excluir correlación consigo mismo
                print(f"  - {feature}: {corr:.2f}")
        
        print("\nCorrelación media (0.25-0.5):")
        for feature, corr in medium_corr.items():
            print(f"  - {feature}: {corr:.2f}")
        
        print("\nBaja correlación (<0.25):")
        for feature, corr in low_corr.items():
            print(f"  - {feature}: {corr:.2f}")
        
        return laptime_corr
  



In [ ]:
# Ejecutar análisis de correlaciones con LapTime
laptime_correlations = analyze_laptime_correlations(correlation_matrix)

In [ ]:
# Celda 5: Recomendar conjunto final de características
def recommend_final_features(corr_with_laptime, high_corr_pairs):
    print("\nRecomendaciones para el conjunto final de características:")
    
    # 1. Identificar características altamente predictivas
    if corr_with_laptime is not None:
        high_pred_features = corr_with_laptime[corr_with_laptime >= 0.5].index.tolist()
        if 'LapTime' in high_pred_features:
            high_pred_features.remove('LapTime')  # Eliminar la variable objetivo
        
        print("1. Características altamente predictivas a mantener:")
        for feature in high_pred_features:
            print(f"   - {feature}")
    
    # 2. Identificar redundancias a eliminar
    if high_corr_pairs:
        print("\n2. Redundancias a considerar (mantener solo una de cada par):")
        for col_i, col_j, _ in high_corr_pairs:
            # Decidir cuál mantener basado en correlación con LapTime
            if corr_with_laptime is not None:
                if corr_with_laptime.get(col_i, 0) >= corr_with_laptime.get(col_j, 0):
                    print(f"   - Mantener {col_i}, considerar eliminar {col_j}")
                else:
                    print(f"   - Mantener {col_j}, considerar eliminar {col_i}")
            else:
                print(f"   - Considerar mantener solo una de: {col_i} o {col_j}")
    
    # 3. Características con valor estratégico pero baja correlación
    if corr_with_laptime is not None:
        strategic_features = ['Position', 'TrackStatus']
        present_strategic = [f for f in strategic_features if f in corr_with_laptime.index]
        
        if present_strategic:
            print("\n3. Características con valor estratégico a mantener a pesar de baja correlación:")
            for feature in present_strategic:
                print(f"   - {feature}: {corr_with_laptime.get(feature, 0):.2f}")
    
    print("\nBalancear estos factores proporcionará un conjunto óptimo de características para el modelo final.")



In [ ]:
# Ejecutar recomendación final
recommend_final_features(laptime_correlations, high_correlation_pairs)

### Feature Selection Analysis Based on Correlation Matrix

Based on the correlation matrix analysis of our cleaned dataset, we've identified several key insights for model optimization:

1. **Highly Predictive Features**:
   - Variables with the strongest correlation to lap time should form the foundation of our predictive model.
   - This includes fuel load, speed measurements at key track sections, and tyre age.

2. **Redundancy Management**:
   - Several pairs of features show extremely high correlation (>0.95), indicating redundancy.
   - For each redundant pair, we prioritize keeping the feature with stronger correlation to lap time.
   - LapNumber and FuelLoad represent the same information in different forms - we retain both for their distinct interpretative value.

3. **Strategic Variables**:
   - Some variables like Position show lower statistical correlation but provide critical strategic insights.
   - These are retained despite lower predictive power due to their importance for race strategy decisions.

4. **Pitstop-related Features**:
   - Pitstop timing and impact variables provide crucial information for strategy modeling.
   - We created composite variables where appropriate to reduce dimensionality while preserving information.

This optimization approach balances statistical considerations with domain knowledge to create a feature set that maximizes both predictive power and strategic insight for F1 strategy decision-making.

In [ ]:
# Mostrar estadísticas descriptivas del dataset final
print("\nEstadísticas descriptivas del dataset final:")
display(clean_data.describe())

# Guardar datos procesados para modelos
# final_model_data.to_csv('data/processed/model_ready_data.csv', index=False)
print("\nDatos listos para modelado.")

In [ ]:
model_data_csv = clean_data.to_csv("../outputs/week3/model_lap_prediction.csv")

## Feature Engineering and Data Preparation

Our data preparation process follows a systematic approach broken down into specialized components:

1. **Data Cleaning**: We convert temporal data to seconds for analysis and remove outliers based on 5th and 95th percentiles to ensure our model isn't trained on anomalous lap times caused by safety cars, accidents, or data errors.

2. **Tire Performance Features**: We create features that capture tire degradation and its effect on performance, including tire age and position changes between laps.

3. **Race Strategy Features**: These features represent strategic elements like DRS usage, undercut opportunities, and gaps to the race leader that influence driving approach and lap times.

4. **Pit Stop Analysis**: We process pit stop data to calculate time lost during stops, stint length, and compound change effects that are critical for strategy modeling.

5. **Feature Selection**: Finally, we identify the most relevant variables for our model based on domain knowledge of Formula 1 racing, ensuring we include key performance drivers while avoiding redundant or noisy features.

This modular approach allows us to clearly understand each transformation and facilitates future refinements to specific aspects of data preparation.

## 7. Análisis Exploratorio de Datos (EDA)

In [ ]:
# Resumen estadístico
print("Resumen estadístico de variables numéricas:")
display(clean_data.describe())

In [ ]:
# Tiempo por vuelta según tipo de neumático
if 'Compound' in clean_data.columns:
    plt.figure(figsize=(12, 6))
    sns.boxplot(x='Compound', y='LapTime', data=clean_data)
    plt.title('Tiempo por Vuelta según Tipo de Neumático')
    plt.xlabel('Compuesto')
    plt.ylabel('Tiempo (segundos)')
    plt.savefig('../outputs/week3/laptime_by_tyre.png')
    plt.show()

In [ ]:
# Diagrama de dispersión: Variables importantes vs LapTime
fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

if 'TyreAge' in pitstop_data.columns:
    sns.scatterplot(x='TyreAge', y='LapTime', hue='Compound', data=pitstop_data, ax=axes[0])
    axes[0].set_title('LapTime vs TyreAge')
    
if 'TrackTemp' in pitstop_data.columns:
    sns.scatterplot(x='TrackTemp', y='LapTime', data=pitstop_data, ax=axes[1])
    axes[1].set_title('LapTime vs TrackTemp')
    
if 'FuelLoad' in pitstop_data.columns:
    sns.scatterplot(x='FuelLoad', y='LapTime', data=pitstop_data, ax=axes[2])
    axes[2].set_title('LapTime vs FuelLoad')

if 'Position' in pitstop_data.columns:
    sns.scatterplot(x='Position', y='LapTime', data=pitstop_data, ax=axes[3])
    axes[3].set_title('LapTime vs Position')
    
plt.tight_layout()
plt.savefig('../outputs/week3/key_variables_scatter.png')
plt.show()

## 7.1 Análisis Específico de Paradas en Boxes

In [ ]:
def visualize_pitstop_impact(data):
    """
    Visualiza el impacto de las paradas en los tiempos por vuelta
    Versión simplificada que asegura generar todos los gráficos relevantes
    """
    # Verificar disponibilidad de datos de paradas
    has_pitstop_data = ('LapsSincePitStop' in data.columns and 'PitNextLap' in data.columns)
    
    if not has_pitstop_data:
        print("No hay suficientes datos de paradas disponibles para visualización.")
        return
    
    # 1. Degradación de neumáticos por compuesto
    plt.figure(figsize=(14, 8))
    
    # Filtrar para mostrar hasta 20 vueltas después de una parada
    plot_data = data[data['LapsSincePitStop'] <= 20].copy()
    
    # Verificar si hay datos suficientes
    if len(plot_data) < 10:
        print("Datos insuficientes para analizar degradación por compuesto.")
    else:
        # Agrupar por tipo de compuesto y vueltas desde parada
        plot_data['LapsSincePitStop'] = plot_data['LapsSincePitStop'].astype(int)
        grouped = plot_data.groupby(['Compound', 'LapsSincePitStop'])['LapTime'].mean().reset_index()
        
        # Dibujar una línea por cada compuesto
        for compound in grouped['Compound'].unique():
            compound_data = grouped[grouped['Compound'] == compound]
            color = compound_colors.get(compound, 'black')
            plt.plot(compound_data['LapsSincePitStop'], 
                     compound_data['LapTime'], 
                     'o-', 
                     color=color,
                     label=f'Compuesto {compound}')
        
        plt.xlabel('Vueltas desde la parada')
        plt.ylabel('Tiempo por vuelta promedio (s)')
        plt.title('Degradación de neumáticos por compuesto')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.savefig('../outputs/week3/tyre_degradation_by_compound.png')
        plt.show()
    
    # 2. Análisis de paradas por compuesto
    if 'PitNextLap' in data.columns:
        # Identificar vueltas previas a paradas
        pitstop_rows = data['PitNextLap'] == 1
        pitstop_count = pitstop_rows.sum()
        
        if pitstop_count > 0:
            print(f"Encontradas {pitstop_count} filas que indican paradas.")
            pitstop_data = data[pitstop_rows]
            
            # 2.1 Contar paradas por cada piloto para verificar
            pit_by_driver = pitstop_data.groupby('Driver').size()
            print("\nNúmero de paradas por piloto:")
            print(pit_by_driver)
            
            # 2.2 Gráfico de compuestos usados antes de paradas
            plt.figure(figsize=(10, 6))
            compound_counts = pitstop_data['Compound'].value_counts()
            ax = sns.barplot(x=compound_counts.index, y=compound_counts.values, 
                           palette={comp: compound_colors.get(comp, 'gray') for comp in compound_counts.index})
            
            # Añadir etiquetas
            for i, count in enumerate(compound_counts):
                ax.text(i, count/2, str(count), ha='center', va='center', fontweight='bold')
                
            plt.title('Compuestos utilizados antes de paradas')
            plt.xlabel('Compuesto')
            plt.ylabel('Número de paradas')
            plt.savefig('../outputs/week3/compounds_before_pitstop.png')
            plt.show()
        else:
            print("No se encontraron filas que indiquen paradas en boxes (PitNextLap=1).")
    else:
        print("No se encuentra la columna 'PitNextLap' para analizar paradas.")

# Ejecutar con los datos limpios
visualize_pitstop_impact(final_model_data)

## 8. Preprocesamiento para Modelado

In [ ]:
# Sección previa a "8. Preprocesamiento para Modelado"
# Finalizar la selección de variables basada en el análisis de correlación

# 1. Definir qué variables mantener según el análisis previo
variables_to_keep = [
    # Variables con alta correlación con LapTime
    'LapTime', 'FuelLoad', 'TyreAge', 'SpeedI1', 'SpeedI2',
    # Variables estratégicas
    'Position', 'Compound', 'LapsSincePitStop',
    # Información del piloto
    'Driver'
]

# 2. Verificar cuáles de estas variables están disponibles
available_vars = [var for var in variables_to_keep if var in clean_data.columns]
print(f"Variables finales seleccionadas para modelado ({len(available_vars)}/{len(variables_to_keep)}):")
for var in available_vars:
    print(f"- {var}")

# 3. Crear DataFrame final para modelado
model_ready_data = clean_data[available_vars].copy()

# 4. Guardar el dataset listo para modelado
model_ready_data.to_csv('../outputs/week3/model_ready_data.csv', index=False)
print(f"\nDatos listos para modelado guardados con {model_ready_data.shape[0]} filas y {model_ready_data.shape[1]} columnas")

# 5. Verificar valores faltantes finales
missing_values = model_ready_data.isnull().sum()
if missing_values.sum() > 0:
    print("\nValores faltantes en el dataset final:")
    print(missing_values[missing_values > 0])
else:
    print("\nNo hay valores faltantes en el dataset final.")

In [ ]:
## 8. Preprocesamiento para Modelado

def preprocess_data_for_modeling(df):
    """
    Preprocesa los datos para el modelado, dividiendo en conjuntos de entrenamiento/prueba
    y configurando los transformadores necesarios.
    
    Args:
        df: DataFrame listo para modelado
    
    Returns:
        X_train, X_test, y_train, y_test, preprocessor
    """
    # Separar características y objetivo
    X = df.drop('LapTime', axis=1)
    y = df['LapTime']
    
    # Identificar columnas categóricas y numéricas
    cat_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
    num_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
    
    print(f"Características categóricas: {cat_cols}")
    print(f"Características numéricas: {num_cols}")
    
    # Configurar transformadores para preprocesamiento
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), num_cols),
            ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
        ])
    
    # Dividir datos en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42)
    
    print(f"Conjunto de entrenamiento: {X_train.shape[0]} muestras")
    print(f"Conjunto de prueba: {X_test.shape[0]} muestras")
    
    return X_train, X_test, y_train, y_test, preprocessor

# Preprocesar datos para modelado
X_train, X_test, y_train, y_test, preprocessor = preprocess_data_for_modeling(model_ready_data)

## 9. Entrenamiento de Modelo XGBoost

In [ ]:
def train_xgboost(X_train, X_test, y_train, y_test, preprocessor):
    """
    Entrena un modelo XGBoost
    """
    # Crear pipeline con preprocesamiento y modelo
    model = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', xgb.XGBRegressor(objective='reg:squarederror'))
    ])
    
    # Parámetros para Grid Search
    param_grid = {
        'regressor__n_estimators': [100, 200],
        'regressor__learning_rate': [0.01, 0.1],
        'regressor__max_depth': [3, 5, 7],
        'regressor__min_child_weight': [1, 3]
    }
    
    # Grid Search
    grid_search = GridSearchCV(
        model, param_grid, cv=3, scoring='neg_mean_squared_error', n_jobs=-1
    )
    
    # Entrenar modelo
    print("Entrenando modelo XGBoost con GridSearchCV (esto puede tardar varios minutos)...")
    grid_search.fit(X_train, y_train)
    
    # Mejores parámetros
    print("\nMejores parámetros XGBoost:")
    for param, value in grid_search.best_params_.items():
        print(f"  {param}: {value}")
    
    # Predecir
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)
    
    # Evaluar
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    print(f"\nMétricas de evaluación del modelo XGBoost:")
    print(f"  MSE: {mse:.4f}")
    print(f"  RMSE: {rmse:.4f} segundos")
    print(f"  MAE: {mae:.4f} segundos")
    print(f"  R²: {r2:.4f}")
    
    # Analizar características importantes
    if hasattr(best_model.named_steps['regressor'], 'feature_importances_'):
        # Obtener nombres de características después del preprocesamiento
        try:
            cat_cols = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
            num_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
            
            # Obtener nombres de características después de one-hot encoding
            encoder = best_model.named_steps['preprocessor'].transformers_[1][1].named_steps['onehot']
            cat_features = encoder.get_feature_names_out(cat_cols).tolist()
            all_features = num_cols + cat_features
            
            # Crear DataFrame con importancias
            importances = best_model.named_steps['regressor'].feature_importances_
            feature_importance = pd.DataFrame({'Feature': all_features, 'Importance': importances})
            feature_importance = feature_importance.sort_values('Importance', ascending=False).head(15)
            
            # Visualizar
            plt.figure(figsize=(12, 8))
            sns.barplot(x='Importance', y='Feature', data=feature_importance)
            plt.title('Top 15 Características Más Importantes - XGBoost')
            plt.tight_layout()
            plt.savefig('outputs/week3/xgboost_feature_importance.png')
            plt.show()
        except Exception as e:
            print(f"Error al analizar importancia de características: {e}")
    
    return best_model, y_pred

# Entrenar modelo XGBoost
xgb_model, y_pred_xgb = train_xgboost(X_train, X_test, y_train, y_test, preprocessor)

## 10. Entrenamiento de Red Neuronal (Opcional)

Este paso es opcional y puede omitirse si prefieres usar solo XGBoost.

In [ ]:
def train_pytorch_nn(X_train, X_test, y_train, y_test, preprocessor):
    """
    Entrena un modelo de red neuronal con PyTorch
    """
    # Aplicar preprocesamiento
    print("Preprocesando datos para la red neuronal...")
    X_train_processed = preprocessor.fit_transform(X_train)
    X_test_processed = preprocessor.transform(X_test)
    
    # Convertir a tensores
    X_train_tensor = torch.FloatTensor(X_train_processed.toarray())
    y_train_tensor = torch.FloatTensor(y_train.values).reshape(-1, 1)
    X_test_tensor = torch.FloatTensor(X_test_processed.toarray())
    y_test_tensor = torch.FloatTensor(y_test.values).reshape(-1, 1)
    
    # Crear conjuntos de datos y cargadores
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    
    # Inicializar modelo
    print(f"Inicializando red neuronal con {X_train_processed.shape[1]} entradas...")
    input_size = X_train_processed.shape[1]
    model = LapTimeNN(input_size)
    
    # Definir criterio y optimizador
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Guardar historial de pérdidas para graficar
    losses = []
    
    # Entrenar modelo
    print("\nEntrenando red neuronal...")
    num_epochs = 100
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        epoch_loss = running_loss / len(train_loader)
        losses.append(epoch_loss)
        
        if (epoch+1) % 10 == 0 or epoch == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}')
    
    # Visualizar curva de aprendizaje
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, num_epochs+1), losses)
    plt.xlabel('Época')
    plt.ylabel('Pérdida')
    plt.title('Curva de Aprendizaje - Red Neuronal')
    plt.grid(True)
    plt.savefig('outputs/week3/neural_network_learning_curve.png')
    plt.show()
    
    # Evaluar modelo
    model.eval()
    with torch.no_grad():
        y_pred_tensor = model(X_test_tensor)
        y_pred = y_pred_tensor.numpy().flatten()
        
    # Métricas
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    print(f"\nMétricas de evaluación de la Red Neuronal:")
    print(f"  MSE: {mse:.4f}")
    print(f"  RMSE: {rmse:.4f} segundos")
    print(f"  MAE: {mae:.4f} segundos")
    print(f"  R²: {r2:.4f}")
    
    return model, y_pred

# Preguntar si se desea entrenar la red neuronal
train_nn = True  # Cambiar a False para omitir este paso
if train_nn:
    nn_model, y_pred_nn = train_pytorch_nn(X_train, X_test, y_train, y_test, preprocessor)
else:
    nn_model, y_pred_nn = None, None

## 11. Visualización y Comparación de Resultados

In [ ]:
def visualize_predictions(y_test, y_pred_xgb, y_pred_nn=None, max_points=1000):
    """
    Visualiza los resultados de los modelos
    """
    # Limitar número de puntos para visualización clara
    if len(y_test) > max_points:
        # Muestrear aleatoriamente para mantener la distribución
        indices = np.random.choice(len(y_test), max_points, replace=False)
        y_test_sample = y_test.iloc[indices]
        y_pred_xgb_sample = y_pred_xgb[indices]
        if y_pred_nn is not None:
            y_pred_nn_sample = y_pred_nn[indices]
    else:
        y_test_sample = y_test
        y_pred_xgb_sample = y_pred_xgb
        y_pred_nn_sample = y_pred_nn
    
    # Configurar tamaño de figura
    plt.figure(figsize=(16, 8))
    
    # Determinar número de subplots
    if y_pred_nn is not None:
        n_plots = 3
    else:
        n_plots = 2
    
    # 1. Dispersión XGBoost
    plt.subplot(1, n_plots, 1)
    plt.scatter(y_test_sample, y_pred_xgb_sample, alpha=0.5)
    plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
    plt.xlabel('Tiempo real (s)')
    plt.ylabel('Tiempo predicho (s)')
    plt.title('XGBoost: Predicciones vs Reales')
    
    # 2. Residuos XGBoost
    plt.subplot(1, n_plots, 2)
    residuals_xgb = y_test_sample - y_pred_xgb_sample
    plt.scatter(y_pred_xgb_sample, residuals_xgb, alpha=0.5)
    plt.axhline(y=0, color='r', linestyle='--')
    plt.xlabel('Predicción (s)')
    plt.ylabel('Residuos (s)')
    plt.title('XGBoost: Residuos vs Predicciones')
    
    # 3. Si hay predicciones de red neuronal
    if y_pred_nn is not None:
        plt.subplot(1, n_plots, 3)
        plt.scatter(y_test_sample, y_pred_nn_sample, alpha=0.5)
        plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
        plt.xlabel('Tiempo real (s)')
        plt.ylabel('Tiempo predicho (s)')
        plt.title('Red Neuronal: Predicciones vs Reales')
    
    plt.tight_layout()
    plt.savefig('outputs/week3/prediction_results.png')
    plt.show()
    
    # Comparativa de errores
    if y_pred_nn is not None:
        errors_xgb = np.abs(y_test - y_pred_xgb)
        errors_nn = np.abs(y_test - y_pred_nn)
        
        plt.figure(figsize=(12, 6))
        plt.hist(errors_xgb, alpha=0.5, bins=50, label='XGBoost')
        plt.hist(errors_nn, alpha=0.5, bins=50, label='Red Neuronal')
        plt.legend()
        plt.xlabel('Error Absoluto (s)')
        plt.ylabel('Frecuencia')
        plt.title('Distribución de Errores por Modelo')
        plt.xlim(0, min(10, max(errors_xgb.max(), errors_nn.max())))
        plt.savefig('outputs/week3/error_distribution.png')
        plt.show()

# Visualizar resultados
visualize_predictions(y_test, y_pred_xgb, y_pred_nn if train_nn else None)

## 12. Guardar Modelos

In [ ]:
def save_models(xgb_model, nn_model=None):
    """
    Guarda los modelos entrenados
    """
    # Guardar modelo XGBoost
    joblib.dump(xgb_model, 'models/week3/xgboost_laptime.joblib')
    print("Modelo XGBoost guardado en 'models/week3/xgboost_laptime.joblib'")
    
    # Guardar modelo PyTorch si existe
    if nn_model is not None:
        torch.save(nn_model.state_dict(), 'models/week3/nn_laptime.pth')
        print("Modelo PyTorch guardado en 'models/week3/nn_laptime.pth'")

# Guardar modelos
save_models(xgb_model, nn_model if train_nn else None)

## 13. Prueba de Predicción con Nuevos Datos

Probemos el modelo con una situación de carrera hipotética.

In [ ]:
def test_prediction(model, available_features):
    """
    Prueba el modelo con datos hipotéticos
    """
    # Crear un escenario de ejemplo
    example_drivers = ["VER", "HAM", "LEC", "PER", "SAI"]
    compounds = ["SOFT", "MEDIUM", "HARD"]
    
    # Crear DataFrame de ejemplo
    example_data = []
    
    for driver in example_drivers:
        for compound in compounds:
            for tyre_age in [1, 10, 20]:
                # Valores predeterminados
                row = {
                    'Driver': driver,
                    'Compound': compound,
                    'TyreAge': tyre_age,
                    'LapNumber': 30,
                    'TrackTemp': 35.0,
                    'AirTemp': 25.0,
                    'Position': 5,
                    'FuelLoad': 0.5,
                    'LapsSincePitStop': tyre_age,  # Si está disponible
                    'PitNextLap': 0,  # No hay parada en la siguiente vuelta
                }
                
                # Añadir solo características disponibles
                example_row = {k: v for k, v in row.items() if k in available_features}
                example_data.append(example_row)
    
    example_df = pd.DataFrame(example_data)
    
    # Hacer predicciones
    predictions = model.predict(example_df)
    
    # Añadir predicciones al DataFrame
    example_df['PredictedLapTime'] = predictions
    
    # Visualizar resultados
    print("\nPredicciones para escenarios de ejemplo:")
    display(example_df)
    
    # Graficar por compuesto y edad de neumáticos
    if 'Compound' in example_df.columns and 'TyreAge' in example_df.columns:
        plt.figure(figsize=(12, 8))
        for driver in example_drivers[:3]:  # Limitar a 3 pilotos para claridad
            driver_data = example_df[example_df['Driver'] == driver]
            for compound in compounds:
                tyre_data = driver_data[driver_data['Compound'] == compound]
                plt.plot(tyre_data['TyreAge'], tyre_data['PredictedLapTime'], 
                         marker='o', linestyle='-', label=f"{driver} - {compound}")
                
        plt.xlabel('Edad de Neumáticos (vueltas)')
        plt.ylabel('Tiempo Predicho (s)')
        plt.title('Predicción de Tiempos por Tipo y Edad de Neumáticos')
        plt.legend()
        plt.grid(True)
        plt.savefig('outputs/week3/tyre_age_predictions.png')
        plt.show()
    
    return example_df

# Probar predicciones con el modelo XGBoost
try:
    prediction_examples = test_prediction(xgb_model, available_features)
except Exception as e:
    print(f"Error al realizar predicciones de prueba: {e}")

## 14. Conclusiones y Próximos Pasos

### Conclusiones

En este notebook, hemos:
1. Cargado y preparado datos de FastF1 y OpenF1, incluyendo información de vueltas, paradas y condiciones meteorológicas
2. Realizado feature engineering para crear características como edad de neumáticos, carga de combustible, y paradas en boxes
3. Analizado el impacto de las paradas y la degradación de neumáticos en los tiempos por vuelta
4. Entrenado un modelo XGBoost para predecir tiempos por vuelta
5. Opcionalmente, entrenado una red neuronal para comparar su rendimiento
6. Visualizado y evaluado los resultados

### Resultados Principales
- El modelo XGBoost puede predecir tiempos por vuelta con un error medio de X segundos
- Las características más importantes para la predicción son [listar las top 3-5 características]
- La degradación de neumáticos tiene un impacto significativo en los tiempos por vuelta
- Las paradas en boxes muestran patrones claros en cuanto a su timing y elección de compuesto

### Próximos Pasos (Semana 4)
1. Integrar este modelo de predicción con un sistema de decisiones basado en reglas
2. Añadir lógica para recomendar estrategias de paradas en boxes
3. Desarrollar sistema para simular undercuts/overcuts basados en las predicciones
4. Crear visualizaciones interactivas para analizar el impacto de diferentes estrategias